# Группировка и агрегирование. Сводные таблицы.

---

**Источники:**

[Сводные таблицы в Python](http://datareview.info/article/svodnyie-tablitsyi-v-python/)

---

## Подготовка окружения

In [ ]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# для Linux
!which pip

# для Windows
# !pip -V

In [ ]:
!conda install pandas -y

In [ ]:
import pandas as pd

pd.__version__

### Группировка и агрегирование в `pandas`

За группировку отвечает метод `pandas.groupby`.

Для примера возьмем [данные (data set) Титаника](https://www.kaggle.com/c/titanic).

In [ ]:
titanic_df = pd.read_csv('./../../data/titanic.csv')

# показать все колонки
pd.options.display.max_columns = None

titanic_df

In [ ]:
# показать первые 5 строк
titanic_df.head()

In [ ]:
# подсчитать, сколько женщин и мужчин выжило, а сколько нет
titanic_df.groupby(['Sex', 'Survived'])['PassengerId'].count()

In [ ]:
# анализ выживания в разрезе класса кабины
titanic_df.groupby(['Pclass', 'Survived'])['PassengerId'].count()

In [ ]:
# процент выживших для каждого пола
titanic_df.groupby('Sex')[['Survived']].mean()

Грубо говоря, из каждых четырех женщин, находившихся на борту, выжили три, в то время как из каждых пяти мужчин выжил только один!

In [ ]:
# взаимосвязь между показателем выживаемости, полом и классом
titanic_df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean').unstack()

In [ ]:
# если без unstack()
titanic_df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean')

Хотя каждый шаг этой последовательности вполне понятен, тем не менее длинную строку кода достаточно трудно читать и использовать.

Подобные операции широко распространены, в связи с чем библиотека `pandas` имеет в своем составе специальный метод `pivot_table,` лаконично реализующий данный тип многомерной агрегации.

## Что такое сводная таблица?

**Сводная таблица (англ. Pivot table)** — инструмент обработки данных, служащий для их обобщения.


*Термин "сводная таблица" может быть знаком из `Microsoft Excel` или любым иным, предназначенным для обработки и анализа данных. 


В `pandas` сводные таблицы строятся через метод `DataFrame.pivot_table`.

In [ ]:
# взаимосвязь между показателем выживаемости, полом и классом
titanic_df.pivot_table(values='Survived', 
                       index='Sex', 
                       columns='Pclass')

In [ ]:
# посчитать сколько всего женщин и мужчин было в конкретном классе корабля
pvt = titanic_df.pivot_table(index=['Sex'], 
                             columns=['Pclass'], 
                             values='Name', 
                             aggfunc='count')

pvt

In [ ]:
# качестве индекса будет пол человека, 
# колонками станут значения из Pclass, 
# функцией агрегирования будет count по колонке Name.
pvt.loc['female', [1, 2, 3]]

### Многоуровневые сводные таблицы

Точно так же, как при использовании `groupby`, группирование в сводной таблице может иметь несколько уровней и задаваться посредством различных параметров.

In [ ]:
# разделить возраст на интервалы
age = pd.cut(titanic_df['Age'], [0, 18, 80])

age

In [ ]:
# взаимосвязь между 
# показателем выживаемости, полом, классом и возрастом

titanic_df.pivot_table(values='Survived', 
                       index=['Sex', age], 
                       columns='Pclass')

In [ ]:
# добавить информацию о стоимости билета (квантили):
fare = pd.qcut(titanic_df['Fare'], 2)

fare

In [ ]:
titanic_df.pivot_table(values='Survived', 
                       index=['Sex', age], 
                       columns=[fare, 'Pclass'])

В результате получили четырехмерную агрегацию, демонстрирующую взаимосвязь между соответствующими величинами.

### Дополнительные параметры сводной таблицы

Полная [сигнатура метода `pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) объекта `DataFrame` является следующей:

```
pandas.pivot_table(values=None, 
                   index=None, 
                   columns=None, 
                   aggfunc='mean', 
                   fill_value=None, 
                   margins=False, 
                   dropna=True, 
                   margins_name='All', 
                   observed=False)
```


Параметры `fill_value` и `dropna` задают способ обработки отсутствующих данных. 

Параметр `aggfunc` задает тип агрегации. По умолчанию его значение равно `mean`. Как и в случае `groupby`, тип агрегации можно задать либо с помощью предопределенной строки (например, `sum`, `mean`, `count`, `min`, `max` и др.), либо посредством функции, реализующей агрегацию (например, `np.sum()`, `min()`, `sum()` и др.).
Кроме того, параметр `aggfunc` может быть задан в виде словаря, отображающего столбцы на любые из желаемых значений, перечисленных выше:

In [ ]:
titanic_df.pivot_table(index='Sex', 
                       columns='Pclass', 
                       aggfunc={'Survived': sum, 
                                'Fare': 'mean'})